In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy

matplotlib.rc('xtick', labelsize=18)     
matplotlib.rc('ytick', labelsize=18)

In [21]:
def plot_cer_co2(R, number):
    
    ''' Plot of CER (CO2 Evolution Rate) and CO2 against time
    
    Args: 
        R: Data frame of specific reactor ex R1, R2 .. 
        number: Number (integer) of the reactor '''
    
    plt.figure(1,figsize=(16, 14))
    ax1 = plt.subplot(212)
    plt.scatter(R['Time (min)']/60, R['Bioreactor ' + str(number) + ' - CER'])
    plt.setp(ax1.get_xticklabels(), fontsize=6)
    plt.xlabel('Time (hour)', fontsize = 20)
    plt.ylabel('CER (CO2 % / min)', fontsize = 20)
    plt.tick_params(labelsize=15)
    plt.xticks(np.arange(0, len(R), step=5))

    ax2 = plt.subplot(211, sharex=ax1)
    plt.scatter(R['Time (min)']/60, R['Bioreactor ' + str(number) + ' - Off-gas CO2%'])
    plt.setp(ax2.get_xticklabels(), visible=False)
    plt.ylabel('CO2 %', fontsize = 20)
    plt.title('Reactor ' + str(number), fontsize = 20)
    plt.tick_params(labelsize=15)

    plt.show()

In [22]:
def calculate_mu(R, number):
    
    ''' Resets time in data frame to where CER begins,
    and calculates mu (growth rate) from CER. 
    
    Args: 
        R: Data frame of specific reactor ex R1, R2 .. 
        number: Number (integer) of the reactor
        
    Returns: 
        data_frame_selected_values: A data frame that starts from time 0 when CER begins
        mu: The calculated growth rate from CER'''
    
    # Shorten dataframe, so we only have values from where CER values begin
    data_frame_selected_values = R[np.isfinite(R['Bioreactor ' + str(number) + ' - CER'])]
    data_frame_selected_values.reset_index(inplace=True, drop=True)
    
    # Reset the time, so the first values corresponds to time 0
    data_frame_selected_values = data_frame_selected_values.copy()
    data_frame_selected_values['Time (min)'] = data_frame_selected_values['Time (min)']-data_frame_selected_values['Time (min)'][0]

    tCER = []
    tCER.append(0)  # Here set the initial value of tCER if you have that.

    for i in range(0, (len(data_frame_selected_values['Time (min)']) - 1)):

        tCER_i = ((data_frame_selected_values['Bioreactor ' + str(number) + ' - CER'][i] + data_frame_selected_values['Bioreactor ' + str(number) + ' - CER'][i + 1]) / 2) * (data_frame_selected_values['Time (min)'][i + 1] - data_frame_selected_values['Time (min)'][i]) + tCER[i] # [CO2 %]  
        tCER.append(tCER_i) 

    mu = data_frame_selected_values['Bioreactor ' + str(number) + ' - CER'] / tCER  # [1/h]
    
    return data_frame_selected_values, mu

In [ ]:
def calculate_mu_fedbatch(R, number):
    
    ''' Resets time in data frame to where CER begins,
    and calculates mu (growth rate) from CER. 
    
    Args: 
        R: Data frame of specific reactor ex R1, R2 .. 
        number: Number (integer) of the reactor
        
    Returns: 
        data_frame_selected_values: A data frame that starts from time 0 when CER begins
        mu: The calculated growth rate from CER'''
    
    # Shorten dataframe, so we only have values from where CER values begin
    data_frame_selected_values = R[np.isfinite(R['Bioreactor ' + str(number) + ' - CER'])]
    data_frame_selected_values.reset_index(inplace=True, drop=True)
    
    # Reset the time, so the first values corresponds to time 0
    #data_frame_selected_values = data_frame_selected_values.copy()
    #data_frame_selected_values['Time (min)'] = data_frame_selected_values['Time']-data_frame_selected_values['Time'][0]

    tCER = []
    tCER.append(0)  # Here set the initial value of tCER if you have that.

    for i in range(0, (len(data_frame_selected_values['Time']) - 1)):

        tCER_i = ((data_frame_selected_values['Bioreactor ' + str(number) + ' - CER'][i] + data_frame_selected_values['Bioreactor ' + str(number) + ' - CER'][i + 1]) / 2) * (data_frame_selected_values['Time'][i + 1] - data_frame_selected_values['Time'][i]) + tCER[i] # [CO2 %]  
        tCER.append(tCER_i) 

    mu = data_frame_selected_values['Bioreactor ' + str(number) + ' - CER'] / tCER  # [1/h]
    
    return data_frame_selected_values, mu

In [24]:
def plot_mu(data_frame_selected_values, mu):
    
    ''' Plot of mu (growth rate) against time
    
    Args: 
        data_frame_selected_values: Data frame of specific reactor with altered time period
        mu: The calculated mu (growth rate) '''
    
    plt.figure(1,figsize=(16, 7))
    plt.scatter(data_frame_selected_values['Time (min)']/60, mu*60)
    plt.ylabel('mu (1/hour)', fontsize = 20)
    plt.xlabel('Time (hour)', fontsize = 20)
    plt.tick_params(labelsize=15)
    plt.xticks(np.arange(0, 120, step=5))


    plt.show()

In [25]:
def plot_mu_smaller_interval(data_frame_selected_values, mu):

    ''' Plot of mu (growth rate) against time in a smaller range
    
    Args: 
        data_frame_selected_values: Data frame of specific reactor with altered time period
        mu: The calculated mu (growth rate) '''
        
    plt.figure(1,figsize=(16, 7))
    plt.scatter(data_frame_selected_values['Time (min)'][0:40]/60, mu[0:40]*60)
    plt.ylabel('mu (1/hour)', fontsize = 20)
    plt.xlabel('Time (hour)', fontsize = 20)
    plt.tick_params(labelsize=15)


    plt.show()

In [1]:
def time_converter(sample_timepoints):
    
    ''' The time of where the samples are taken, is being converted into decimals
    
    Args: 
        sample_timepoints: The sample timepoints
        
    Returns: 
        time_hours: The timepoints in decimals of reactor 1-8 in a data frame '''
    
    sample_timepoints = pd.ExcelFile(sample_timepoints)
    sample_timepoints = sample_timepoints.parse('Sheet1')


    # The batches from all experiments starts at 14:30, therefore this fits with all the data. 
    # There is just the need to add 1.30 minutes to each experiment. 
    
    # Create a dataframe to put the result in (which is the converted time to decimals)
    sample_timepoints_decimals = pd.DataFrame(columns=['R1 Time (hours)'])


    # This loops through all datapoints, converts it string, splits and calculate the time to minutes in decimals
    for i in range(len(sample_timepoints)):
        t = sample_timepoints['timepoints'][i]
        t = str(t)
        (h, m, s) = t.split(':')
        result = int(h) * 60 + int(m) + int(s) / 60
        sample_timepoints_decimals.loc[
        i, ['R1 Time (hours)']] = result 

    # To get hours instead and add 24 hours to next day
    time_hours = sample_timepoints_decimals/60
    next_day = time_hours[8:11] + 24
    time_hours = time_hours.drop(time_hours.tail(3).index,inplace=False)

    # I substract 9.5 so it fits with the online modified data. This is because the time were CER starts at was 9:30 the day where the samples were taken.
    time_hours = time_hours.append(next_day, ignore_index=True)
    time_hours = time_hours-9.5
    
    # Add 1.30 minutes for every experiment (reactor)
    for i in range(2,9):
    
        add_hours = 1/60 + 1/120
        sample_timepoints_R2 = time_hours.iloc[:,-1] + add_hours
        time_hours['R' + str(i) + ' Time (hours)'] = sample_timepoints_R2
    
    
    return time_hours

In [29]:
def time_converter_R9_to_R12(sample_timepoints):
    
    ''' The time of where the samples are taken, is being converted into decimals
    
    Args: 
        sample_timepoints: The sample timepoints
        
    Returns: 
        time_hours: The timepoints in decimals of reactor 9-12 in a data frame '''
    
    sample_timepoints = pd.ExcelFile(sample_timepoints)
    sample_timepoints = sample_timepoints.parse('Sheet1')

    # Because all data sets starts from 14:30, the method can be used for all data sets
    # There is just the need to add 1.30 minutes to each experiment. 
    
    # Create a dataframe to put the result in (which is the converted time to decimals)
    sample_timepoints_decimals = pd.DataFrame(columns=['R9 Time (hours)'])

    # This loops through all datapoints,
    # convert it string, splits and calculate the time to minutes in decimals
    for i in range(len(sample_timepoints)):
        t = sample_timepoints['timepoints'][i]
        t = str(t)
        (h, m, s) = t.split(':')
        result = int(h) * 60 + int(m) + int(s) / 60
        sample_timepoints_decimals.loc[
        i, ['R9 Time (hours)']] = result 

    # To get hours instead and add 24 hours to get to the next day
    time_hours = sample_timepoints_decimals/60
    next_day = time_hours[7:18] + 24
    time_hours = time_hours.drop(time_hours.tail(11).index,inplace=False)
    time_hours = time_hours.append(next_day, ignore_index=True)

    # And add 24 to the 3 day
    next_day = time_hours[17:18] + 24
    time_hours = time_hours.drop(time_hours.tail(1).index,inplace=False)
    time_hours = time_hours.append(next_day, ignore_index=True)
    time_hours = time_hours-9.5
    
    # Add 1.30 minutes for every experiment (reactor)
    for i in range(10,13):
    
        add_hours = 1/60 + 1/120
        sample_timepoints_R2 = time_hours.iloc[:,-1] + add_hours
        time_hours['R' + str(i) + ' Time (hours)'] = sample_timepoints_R2
        
    return time_hours

In [28]:
def plot_biomass(sample_timepoints_R, biomass_R, plot_biomass_R):
        
    ''' A plot of the biomass against time, and a png file saved to the computer of the plot
    
    Args: 
        sample_timepoints_R: The sample timepoints
        biomass_R: The biomass from the experiments
        plot_biomass_R: What the file should be named (string)'''

    plt.figure(1,figsize=(13, 7))
    plt.scatter(sample_timepoints_R,biomass_R)
    plt.ylabel('Biomass (g DW/L)', fontsize = 25)
    plt.xlabel('Time (hour)', fontsize = 25)
    plt.title('Biomass',fontsize = 30, weight = 'bold')
    plt.savefig('figures/biomass_plots/' + plot_biomass_R + '.pdf')
    
    plt.show()

In [30]:
def semilog_plot_biomass(sample_timepoints_R, biomass_R):
    
    ''' A plot of the biomass against time
    
    Args: 
        sample_timepoints_R: The sample timepoints
        biomass_R: The biomass from the experiments'''
    
    # Take the logarithmic of biomass, np.log is the same as ln 
    plt.figure(1,figsize=(13, 7))
    plt.scatter(sample_timepoints_R,np.log(biomass_R))
    plt.ylabel('ln(Biomass (g DW/L))', fontsize = 22)
    plt.xlabel('Time (hour)', fontsize = 22)
    plt.title('Semilogarithmic plot of biomass',fontsize = 27, weight = 'bold')
    
    
    plt.show()

In [31]:
def lin_reg_slope_mu(sample_timepoints_R, biomass_R, biomass_R_exponential_phase, sample_timepoints_R_exponential_phase, plot_biomass_R):
    
    ''' Linear regression of the exponential phase of biomass, which prints out the slope and R^2 value and the corresponding plot,
        and a png file saved to the computer of the plot. 
    
    Args: 
        sample_timepoints_R: The sample timepoints
        biomass_R: The biomass from the experiments
        biomass_R_exponential_phase: The biomass data from the exponential phase
        sample_timepoints_R_exponential_phase: The timepoints from the exponential phase'''
    
    x = (sample_timepoints_R_exponential_phase).astype(float)
    y = (biomass_R_exponential_phase).astype(float)

    biomassexp = pd.DataFrame()
    biomassexp['x'] = x
    biomassexp['y'] = y
    biomassexp = biomassexp.dropna()

    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(biomassexp['x'],biomassexp['y'])

    plt.figure(1,figsize=(13, 7))
    plt.plot(sample_timepoints_R, np.log(biomass_R), 'o', label='original data')
    plt.plot(biomassexp['x'], intercept + slope*biomassexp['x'], 'r', label='fitted line')
    plt.ylabel('ln(Biomass (g DW/L))', fontsize = 25)
    plt.xlabel('Time (hour)', fontsize = 25)
    plt.title('ln(X) vs time and regression line',fontsize = 30, weight = 'bold')
    plt.legend(fontsize = 25)
    
    plt.savefig('figures/semlog_biomass_plots/semlog_' + plot_biomass_R + '.pdf')
    
    plt.show()
    
    print('mu_max from biomass (slope):', slope, 'R^2:', r_value)

In [37]:
def add_biomass_time(OD_R, R_compounds, number):
    
    ''' OD is converted into biomass and sample time and biomass are added to the data frame with analytical data
    
    Args: 
        OD_R: OD and time values
        R_compounds: The data frame of the analytical data
        number: The number of the reactor
        
    Returns: 
        time_hours: Data frame with analytical compounds, biomass and sample time '''
    
    R_compounds = R_compounds.drop(['Reactor_sample'], axis = 1)
    R_compounds = R_compounds.reset_index(drop = True)
    
    # This loads the specific sheet and only collects OD where there are values of them
    OD_R = pd.ExcelFile(OD_R)
    OD_R = OD_R.parse('OD_R' + str(number))
    OD_R = OD_R[np.isfinite(OD_R['OD'])]
    OD_R = OD_R.reset_index(drop = True)

    # Put the two data frames together
    R_compounds = pd.concat([OD_R,R_compounds], axis = 1)
    # Converts OD to biomass
    R_compounds['Biomass, g/L'] = R_compounds['OD']*0.38
    
    return R_compounds

In [2]:
def plot_compounds(R, number):
    
    ''' Makes a plot of glucose, acetate, serine and biomass
    
    Args: 
        R: Data frame with the analytical data
        number: Reactor number
         '''

    fig = plt.figure(1,figsize=(12, 6))

    ax1 = fig.add_subplot(111)
    ax1.scatter(R['Time (hours)'],R['Glucose g/L'], linewidths = 5)
    ax1.scatter(R['Time (hours)'],R['Acetate g/L'], linewidths = 2)
    ax1.scatter(R['Time (hours)'],R['Serine g/L'], linewidths = 3)
    ax1.set_ylabel('Glu, Ace, Ser in (g/L)', fontsize = 25)
    plt.xlabel('Time (hour)', fontsize = 25)
    plt.legend(['Glucose', 'Acetate', 'Serine','Biomass'], loc = 2, edgecolor = 'black', fontsize = 20)
    plt.title('Compounds in reactor', fontsize = 30)

    ax2 = ax1.twinx()
    ax2.scatter(R['Time (hours)'], R['Biomass g/L'], c = 'r', linewidths = 5)
    ax2.set_ylabel('Biomass (g/L)', fontsize = 25)
    plt.legend(['Biomass'], loc = 'lower right', edgecolor = 'black', fontsize = 20)
    
    plt.savefig('figures/analytical_data/batch/R' + str(number) + '.pdf')

    plt.show()

In [ ]:
def plot_compounds_micro(R, number):
    
    ''' Makes a plot of glucose, acetate, serine and biomass
    
    Args: 
        R: Data frame with the analytical data
        number: Reactor number
         '''

    fig = plt.figure(1,figsize=(12, 6))

    ax1 = fig.add_subplot(111)
    ax1.scatter(R['Time (hours)'],R['Glucose µg/µL'], linewidths = 5)
    ax1.scatter(R['Time (hours)'],R['Acetate µg/µL'], linewidths = 2)
    ax1.scatter(R['Time (hours)'],R['Serine g/L'], linewidths = 3)
    ax1.set_ylabel('Glu, Ace, Ser in (g/L)', fontsize = 25)
    plt.xlabel('Time (hour)', fontsize = 25)
    plt.legend(['Glucose', 'Acetate', 'Serine','Biomass'], loc = 2, edgecolor = 'black', fontsize = 20)
    plt.title('Compounds in reactor', fontsize = 30)

    ax2 = ax1.twinx()
    ax2.scatter(R['Time (hours)'], R['Biomass g/L'], c = 'r', linewidths = 5)
    ax2.set_ylabel('Biomass (g/L)', fontsize = 25)
    plt.legend(['Biomass'], loc = 'lower right', edgecolor = 'black', fontsize = 20)
    
    plt.savefig('figures/analytical_data/batch/R' + str(number) + '.pdf')

    plt.show()

In [ ]:
def sample_time_fedbatch(hours, minutes, seconds):
    
    ''' Generate sample time with 5 hours invterval in each sample time
    
    Args: 
        R: The sample start time from the experiment
         '''
    
    
    # The CER values starts at 10 hours and since I set that to be 0 in online data, I also have to integrate sample time to fit with that.
    # Therefore I substracted 10 hours. 
    # Then we just add 5 hours between each sample. 
    
    starttime = (hours - 10) + (minutes/60)+seconds/(60*60)
    
    time = pd.Series([starttime])

    for i in range(0, 14):

        add_time = pd.Series([5])
        next_time = time + add_time
        time = time.append(next_time)
        time = time.drop_duplicates()

    return time

In [ ]:
def add_time_bio_ser(R_compounds, reactor_nb, time):
    
    ''' Generate sample time with 5 hours invterval in each sample time
    
    Args: 
        R_compounds: Analytical compounds from experiments
        reactor_nb: The number of the reactor
        time: The sample time
        
    Returns:
        The analytcial compounds with serine, biomass and sample time
         '''
    
    # Adds serine
    R_compounds.reset_index(inplace=True, drop=True)
    serine = 'analytical_data/serine_fedbatch.xlsx'
    serine = pd.ExcelFile(serine)
    serine = serine.parse('R' + str(reactor_nb))
    serine[(serine == 'n.a.')] = np.nan
    R_compounds = R_compounds.copy()
    R_compounds['Serine g/L'] = serine['Conc, g/L']

    # Adds biomass and time
    OD = 'OD/SER_C016_AMBR_ODMeasurements.xlsx'
    OD = pd.ExcelFile(OD)
    OD = OD.parse('R13')
    Biomass = OD['OD']*0.38
    R_compounds['Biomass g/L'] = Biomass
    time.reset_index(inplace=True, drop=True)
    R_compounds['Time (hours)'] = time
    
    return R_compounds

In [ ]:
def batch_amounts(R, reactor_nb, R_volume, a, b, c):

    ''' Calculates amounts of biomass and glucose and saves af csv file with data as well as initial values, time and parameters in volume
    
    Args: 
        R: Data frame with the analytical data in concentration
        reactor_nb: The reactor number
        R_volume: Volume data
        a: Parameter a from volume
        b: Parameter b from volume
        c: Parameter c from volume '''

    R_amounts = pd.DataFrame()
    R_amounts['Time (hours)'] = R['Time (hours)']
    R_amounts['Volume (L)'] = R_volume
    R_amounts['Biomass (g)'] = R['Biomass g/L']*R_volume
    R_amounts['Glucose (g)'] = R['Glucose µg/µL']*R_volume
    R_amounts['Biomass initial (g)'] = R_amounts['Biomass (g)'][0]
    R_amounts['Glucose initial (g)'] = R_amounts['Glucose (g)'][0]
    R_amounts['a'] = a
    R_amounts['b'] = b
    R_amounts['c'] = c

    R_amounts.to_csv('estimation/batch_amounts/R' + str(reactor_nb) + '_amounts.csv')


In [ ]:
def batch_amounts_glucose_grams(R, reactor_nb, R_volume, a, b, c):
    
    ''' Calculates amounts of biomass and glucose and saves af csv file with data as well as initial values, time and parameters in volume
    
    Args: 
        R: Data frame with the analytical data in concentration
        reactor_nb: The reactor number
        R_volume: Volume data
        a: Parameter a from volume
        b: Parameter b from volume
        c: Parameter c from volume'''


    R_amounts = pd.DataFrame()
    R_amounts['Time (hours)'] = R['Time (hours)']
    R_amounts['Volume (L)'] = R_volume
    R_amounts['Biomass (g)'] = R['Biomass g/L']*R_volume
    R_amounts['Glucose (g)'] = R['Glucose g/L']*R_volume
    R_amounts['Biomass initial (g)'] = R_amounts['Biomass (g)'][0]
    R_amounts['Glucose initial (g)'] = R_amounts['Glucose (g)'][0]
    R_amounts['a'] = a
    R_amounts['b'] = b
    R_amounts['c'] = c

    R_amounts.to_csv('estimation/batch_amounts/R' + str(reactor_nb) + '_amounts.csv')


In [ ]:
def plot_compounds_micro_fed(R, number):
    
    ''' Makes a plot of glucose, acetate, serine and biomass from fedbatch
    
    Args: 
        R: Data frame with the analytical data
        number: Reactor number
         '''

    fig = plt.figure(1,figsize=(12, 6))

    ax1 = fig.add_subplot(111)
    ax1.scatter(R['Time (hours)'],R['Glucose µg/µL'], linewidths = 5)
    ax1.scatter(R['Time (hours)'],R['Acetate µg/µL'], linewidths = 2)
    ax1.scatter(R['Time (hours)'],R['Serine g/L'], linewidths = 3)
    ax1.set_ylabel('Glu, Ace, Ser in (g/L)', fontsize = 25)
    plt.xlabel('Time (hour)', fontsize = 25)
    plt.legend(['Glucose', 'Acetate', 'Serine','Biomass'], loc = 2, edgecolor = 'black', fontsize = 20)
    plt.title('Compounds in reactor', fontsize = 30)

    ax2 = ax1.twinx()
    ax2.scatter(R['Time (hours)'], R['Biomass g/L'], c = 'r', linewidths = 5)
    ax2.set_ylabel('Biomass (g/L)', fontsize = 20)
    plt.legend(['Biomass'], loc = 1, edgecolor = 'black', fontsize = 25)
    
    plt.savefig('figures/analytical_data/fed_batch/R' + str(number) + '.pdf')

    plt.show()